# Use Vgg16 model with Keras

[State Farm Distracted Driver Detection | Kaggle](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data)

In [ ]:
%pwd

In [1]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '1815' (I am process '1637')
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
DATA_HOME_DIR = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/"

In [ ]:
%cd $DATA_HOME_DIR

In [ ]:
%cd train

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
# create c0 .. c9 in train/valid directory
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [ ]:
from shutil import copyfile

In [ ]:
# copy c?/*.jpg to sample/train/c?/*.jpg
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%cd ../../..

## train data with pre-trained model

In [3]:
import numpy as np
import pandas as pd
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

In [2]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [9]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model


def finetune(model, num_classes):
    # remove last layer
    model.pop()
    # set all layers untrainable.
    for layer in model.layers: layer.trainable=False
    # add new layer
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def get_batches(path, dirname, gen=image.ImageDataGenerator(), shuffle=True,
                batch_size=64, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

def get_batches2(path, dirname, 
                gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08),
                shuffle=True,
                batch_size=64, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [4]:
def norm_input(x): return x/255.0

def get_model_bn_do():
    model = Sequential([
        Lambda(norm_input, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
model = get_model_bn_do()

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 222, 222)  896         lambda_1[0][0]                   
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 32, 222, 222)  64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 220, 220)  9248        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [7]:
batch_size=64
# path = DATA_HOME_DIR
path = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/sample/"

In [10]:
batches = get_batches2(path,'train', batch_size=batch_size)
val_batches = get_batches2(path,'valid', batch_size=batch_size)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [11]:
# train finetuned model(only last layer)
no_of_epochs=10

for epoch in range(no_of_epochs):
    print ("Running epoch: %d" % epoch)
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
    latest_weights_filename = 'ft%d.h5' % epoch
    model.save_weights(latest_weights_filename)

Running epoch: 0
Epoch 1/1
1500/1500 [==============================] - 55s - loss: 3.2726 - acc: 0.2600 - val_loss: 2.7363 - val_acc: 0.1300
Running epoch: 1
Epoch 1/1
1500/1500 [==============================] - 50s - loss: 2.0448 - acc: 0.3807 - val_loss: 2.7595 - val_acc: 0.1390
Running epoch: 2
Epoch 1/1
1500/1500 [==============================] - 49s - loss: 1.5915 - acc: 0.5193 - val_loss: 2.9618 - val_acc: 0.1160
Running epoch: 3
Epoch 1/1
1500/1500 [==============================] - 50s - loss: 1.2528 - acc: 0.5973 - val_loss: 3.1395 - val_acc: 0.1250
Running epoch: 4
Epoch 1/1
1500/1500 [==============================] - 49s - loss: 1.1478 - acc: 0.6360 - val_loss: 3.4506 - val_acc: 0.1530
Running epoch: 5
Epoch 1/1
1500/1500 [==============================] - 49s - loss: 0.9249 - acc: 0.7053 - val_loss: 3.9966 - val_acc: 0.1490
Running epoch: 6
Epoch 1/1
 988/1500 [==================>...........] - ETA: 11s - loss: 0.9402 - acc: 0.7055

KeyboardInterrupt: 

In [6]:
#model.load_weights("ft0.h5")

## test data

In [ ]:
test_batches = get_batches(DATA_HOME_DIR, 'test', batch_size=2*batch_size, class_mode=None)
preds = model.predict_generator(test_batches, test_batches.nb_sample)

Found 79726 images belonging to 1 classes.


In [ ]:
#df = pd.read_csv(DATA_HOME_DIR+"submission2.csv")
#classes = sorted(batches.class_indices, key=batches.class_indices.get)
#preds = df[classes].values

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)
preds = do_clip(preds,0.93)

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission.head()

In [ ]:
submission.to_csv("submission2.csv", index=False)

In [ ]:
FileLink("submission2.csv")